In [ ]:
cd ../../../
./aestas.ps1 build

In [ ]:
#!import privateVar.csx
// use a individual file to store your private data, like:
// #r "C:\\Users\\YourUserName\\Source\\Aestas\\bin\\Release\\net8.0\\aestas.dll"
// #r "C:\\Users\\YourUserName\\Source\\Aestas\\src\\csharp\\Aestas.CSharp\\bin\\Release\\net8.0\\Aestas.CSharp.dll"
// var geminiKey = "Your Api Key";
// var name = "Bot Name";
// var systemInstruction = "Your System Instruction";
using Aestas;
using static Aestas.Prim;
using static Aestas.Core;
using static Aestas.AutoInit;
using static Aestas.Llms.Gemini;
using Aestas.CSharp;
using Microsoft.FSharp.Core;
using Microsoft.FSharp.Collections;
using Microsoft.FSharp.Control;

In [ ]:
var profile = new GProfile(FSharpHelper.MakeSome(geminiKey),
    null,
    [
        new GSafetySetting("HARM_CATEGORY_HARASSMENT", "BLOCK_NONE"),
        new GSafetySetting("HARM_CATEGORY_HATE_SPEECH", "BLOCK_NONE"),
        new GSafetySetting("HARM_CATEGORY_SEXUALLY_EXPLICIT", "BLOCK_NONE"),
        new GSafetySetting("HARM_CATEGORY_DANGEROUS_CONTENT", "BLOCK_NONE")
    ],
    new GenerationConfig(1.02, 4096, 64, 1));
var model = new GeminiLlm(profile, "gemini-1.5-flash-exp-0827");
var bot = new AestasBot() {
    Name = name,
    Model = model,
    SystemInstruction = systemInstruction,
    SystemInstructionBuilder = 
        PipeLineChain<Tuple<AestasBot, StringBuilder>>.FromFunc(x => Builtin.buildSystemInstruction(x.Item1, x.Item2)),
    MessageReplyStrategy = BotMessageReplyStrategy.StrategyReplyAll
};
bot.SetPrefixBuilder(Builtin.buildPrefix);
var display = "";
var virtualDomain = new VirtualDomain(
    FuncConvert.FromFunc<FSharpList<AestasContent>, Unit>(x => {
        display = String.Join(" ", x);
        return FSharpHelper.UnitValue;
    }),
    FuncConvert.FromFunc<ulong, Unit>(x => FSharpHelper.UnitValue),
    new AestasChatMember(1u, name),
    new AestasChatMember(0u, "User"),
    0u, "Test", true);
bot.BindDomain(virtualDomain);

In [ ]:
FSharpAsync.RunSynchronously(virtualDomain.Input(ListModule.OfSeq<AestasContent>([
    AestasContent.NewAestasText("你好")
])), null, null);
System.Console.WriteLine(display);
foreach (var msg in virtualDomain.Messages)
{
    System.Console.WriteLine($"{msg.Parse().sender.name}: {msg.Preview}");
}